In [ ]:
import os
import sys
import fnmatch
import numpy as np
import pandas as pd
import json
import gzip
import pickle
import csv
import scipy.sparse
Xauth = None
from collections import defaultdict

In [ ]:
with gzip.open('useful_venue_list.pkl.gz','rb') as fp:
    all_venues = pickle.load(fp)
with gzip.open('useful_authors_list.pkl.gz','rb') as fp:
    all_authors = pickle.load(fp)
with gzip.open('useful_papers.pkl.gz','rb') as fp:
    all_papers = pickle.load(fp)

In [ ]:
conf_idx = {v:i for i,v in enumerate(all_venues)}
name_idx = {v:i for i,v in enumerate(all_authors)}
n_confs = len(all_venues)
n_auths = len(all_authors)
print(n_confs,n_auths)

In [ ]:
faculty_affil = pd.read_csv('faculty-affiliations.csv')
ranks = pd.read_csv('other_ranks/ranks.csv')
def csv2dict_str_str(fname):
    with open(fname, mode='r') as infile:
        rdr = csv.reader(infile)
        d = {rows[0].strip(): rows[1].strip() for rows in rdr}
    return d
aliasdict = csv2dict_str_str('dblp-aliases-expanded.csv')

In [ ]:
min_year = all_papers[0][6]
max_year = all_papers[-1][6]
span_years = max_year - min_year + 1
print(min_year,max_year,span_years)

In [ ]:
auth_years = np.ones((n_auths,2)) * np.array([3000,1000]) 
for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    for a in authors:
        i = name_idx[a]
        auth_years[i,0] = min(auth_years[i,0],year)
        auth_years[i,1] = max(auth_years[i,1],year)

In [ ]:
from scipy.stats.mstats import gmean
from scipy.ndimage.filters import gaussian_filter1d
from scipy.stats.mstats import trimmed_mean, trimmed_std

clf_files2 = ['weights_faculty_above6_linear_2_75_25_0.pkl',
              'weights_nsfmarginal_above6_log_2_0_25_0.pkl',
              'weights_salary_above6_linear_2_0_25_0.pkl']
# without salary data
clf_files3 = ['weights_faculty_above6_linear_2_75_25_0.pkl',
              'weights_nsfmarginal_above6_log_2_0_25_0.pkl']

#clf_files2 = [
#              'weights_salary_above6_linear_2_0_25_0.pkl']
clfs2 = [pickle.load(open(c,'rb')) for c in clf_files2]

years_per_conf = clfs2[0].shape[0]//n_confs

#span_years = 50 
YEAR_BLOCKS = span_years//years_per_conf

print(span_years,years_per_conf)
#repeat_needed = span_years//(clfs[0].shape[0]//n_confs)
#if repeat_needed > 1:
#    clfs = [np.repeat(_,repeat_needed) for _ in clfs2]
clfs = clfs2
clfs = [np.squeeze(_) for _ in clfs]


no_conf_weights = abs(np.vstack(clfs)).sum(0)
no_conf_weights = (no_conf_weights != 0).astype(np.float)

print([_.shape for _ in clfs])
clp =  int(os.environ.get('COMBINE_CLIP',14)) #7
clfs2 = []

clfs = [(c-c.mean(0))/c.std(0) for c in clfs]
clfs = [np.minimum(clp*2,np.maximum(-clp*2,c)) for c in clfs]
if False:
    clfs2 = clfs
    pass
elif True:
    for result_clf,w in zip(clfs,[1,1,1]):
        result_clf = result_clf.reshape((-1,years_per_conf))

        #plt.plot(result_clf.sum(0)/result_clf.sum(0).sum(),label='sum')
        print(abs(result_clf.mean(0)).mean(),abs(result_clf.std(0)).mean())
        #result_clf = np.minimum(30,np.maximum(result_clf,-30))
        #result_clf = (result_clf)/result_clf.std(0)
        # nearest reflect mirror
        meanv = gaussian_filter1d(result_clf.mean(0),0.9,mode='nearest')#result_clf.mean(0)#trimmed_mean(result_clf,0,axis=0)#gaussian_filter1d(result_clf.mean(0),0.9,mode='nearest')
        stdv =  gaussian_filter1d(np.sqrt(np.mean((result_clf-meanv)**2,0)),0.9,mode='nearest')#trimmed_std(result_clf,0.0001,axis=0)#gaussian_filter1d(result_clf.std(0),0.9,mode='nearest')
        #plt.plot(stdv)

        result_clf = w*(result_clf-meanv)/stdv
        result_clf = result_clf.reshape((-1))
        clfs2.append(result_clf)
else:
    clfs2 = [(c-c.mean(0))/c.std(0) for c in clfs]
clfs = clfs2

clfs = [np.minimum(clp,np.maximum(-clp,c)) for c in clfs]
clfs = np.vstack(clfs)

if True: # weights
    if False: # per conf
        w = np.exp(-(clfs-np.median(clfs,0))**2/1e-3)
        w /= w.sum(0)
        print(w.sum(1))
    else: # per regression
        w = np.exp(-abs(clfs-np.median(clfs,0)).mean(1)/0.05)
        w /= w.sum()
        w = w.reshape((-1,1))
        print(w)
else:
    w = np.ones(len(clfs2))
#print(w)

clf = np.mean(np.vstack([np.median(clfs,0),w*clfs]),0)*no_conf_weights
#clf = gmean(clfs-clfs.min(0)+1,0)*no_conf_weights
#clf = clf# - clf.min()

clf_gold = np.copy(clf)
np.save('clf_gold.pkl',clf_gold)
#clf_gold = np.ones_like(clf)
#clf = clf_gold
clf_yearsize = np.copy(clf)


In [ ]:
[_ for _ in all_venues if 'Nature'.lower() in _.lower()]

In [ ]:
no_conf_weights[conf_idx['SIGGRAPH']*years_per_conf+(2008-1970)//YEAR_BLOCKS]


In [ ]:
#for paper in all_papers:
#    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
#    n = len(authors)
#    j = years_per_conf*conf_idx[venue] + (year-min_year)//YEAR_BLOCKS
#    clfs[0][conf_idx['SIGGRAPH']*years_per_conf:conf_idx['SIGGRAPH']*years_per_conf]

In [ ]:
#clf_none = np.copy(clf)
#clf_size = np.copy(clf)
#clf_year = np.copy(clf)
#clf_yearsize = np.copy(clf)
clf_none = None

In [ ]:
YEAR_BLOCKS

In [ ]:
if clf_none is not None:
    i = 0
    plt.figure(figsize=(12,12))
    for name,clf in zip(['None','Size','Year','Year + Size'],[clf_none,clf_size,clf_year,clf_yearsize]):
        plt.subplot(2,2,i+1)
        i+=1
        vs = clf.std()
        conf_vector = ['NIPS','AAAI','ICML','IJCAI','AISTATS','UAI']
        #conf_vector = ['SIGIR','JCDL','CIKM','KDD','WWW','SIGMOD Conference']
        #conf_vector = ['STOC','FOCS','SODA','EC','WINE']
        for conf in conf_vector:
            idx = conf_idx[conf]
            _ = plt.plot(np.arange(min_year,max_year+1),(clf[years_per_conf*idx:years_per_conf*(idx+1)]/vs),label=conf,lw=5)
            plt.grid(True)
            plt.xlabel('year')
            plt.ylabel('venue scores (in standard deviations)')
        plt.title(name)
        plt.legend(loc=2,fancybox=True, framealpha=0.8, borderpad=1,frameon=True)
        #plt.ylim(0,25)
        plt.xlim(1970,2020)
        plt.xticks(np.arange(1970,2020,10),[str(_) for _ in np.arange(1970,2020,10)])
        plt.tight_layout()
    plt.suptitle('Normalization Methods',size=36,fontweight='demibold')
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.savefig('size_year_norm2.pdf',facecolor='w',edgecolor='w')
    clf = clf_yearsize
    clf_gold = clf

In [ ]:
import matplotlib.pyplot as plt
for clf_tmp,clf_name in zip(clfs2,clf_files2):
    clf_tmp = clf_tmp.reshape((-1,years_per_conf))[:,-1]
    plt.hist(clf_tmp,100,label=clf_name.split('_')[1],alpha=0.3)
    print(clf_tmp.mean(),clf_tmp.std())
plt.legend()


In [ ]:
if len(clf_files2) == 3:
    df_clfs2 = pd.DataFrame(np.vstack(clfs2).T,columns=['Faculty','NSF','Salary'])
    print(df_clfs2.corr('spearman').to_latex())

In [ ]:
conf_ord = np.argsort(np.squeeze(clf))
#'Scientometrics','JCDL','NIPS',
conf_choice = ['ICLR','ICPR','Foundations and Trends in Electronic Design Automation',
               'FOCS','STOC','ICRA','ICML','SIGGRAPH','HRI','ECCV','Comput. Graph. Forum',
               'Shape Modeling International','Symposium on Geometry Processing',
               'Computer Aided Geometric Design','I. J. Robotics Res.','CVPR',
               'International Journal of Computer Vision','Robotics: Science and Systems',
               'ICRA','WACV','ICML','AISTATS','CoRR','SIGGRAPH Asia','ECCV','ICCV','ISER','MICCAI',
               'Humanoids','3DV','IROS','CoRL','Canadian Conference on AI','ACCV','Graphics Interface',
               'CRV','BMVC','Theory of Computing','SODA','SIAM J. Comput.','J. ACM','NIPS','Allerton',
               'ICC','GLOBECOM','IEEE Trans. Vehicular Technology','IEEE Trans. Wireless Communications','Underwater Networks']
ri_confs = np.zeros(clf_gold.shape[0])
ms = clf_gold.mean()
ss = clf_gold.std()
np.set_printoptions(precision=1)
seen = {}
for i in range(clf_gold.shape[0]):
    idx = conf_ord[-(i+1)]
    conf_name = all_venues[idx//years_per_conf]
    conf_score = clf[idx]
    if conf_name in conf_choice:
        ri_confs[idx] = 1
    if conf_name in conf_choice and conf_name not in seen:
        vec = clfs[:,idx]
        print('{:20s}{}\t{:.1f}\t{}'.format(conf_name[:20],str(min_year + YEAR_BLOCKS*(idx % years_per_conf)),(conf_score-ms)/ss,vec))
        seen[conf_name] =1
ri_confs.shape,ri_confs.sum()
seen = {}
print('\n\n')
conf_ord2 = np.argsort(clf_gold.reshape((-1,years_per_conf))[:,-1])[::-1]
for idx in conf_ord2[:100]:
    conf_name = all_venues[idx]
    conf_score = clf[(idx+1)*years_per_conf-1]
    if True or conf_name in conf_choice:
        vec = clfs[:,(idx+1)*years_per_conf-1]
        print('{:20s}{}\t{:.1f}\t{}'.format(conf_name[:20],2018,(conf_score-ms)/ss,vec))
        seen[conf_name] =1

In [ ]:
if len(clf_files2) == 3:
    clf_fnsf = np.mean(clfs[0:1],0)*no_conf_weights
    clf_fnsf /= clf_fnsf.std()
    clf_s = np.mean(clfs[1:3],0)*no_conf_weights
    clf_s /= clf_s.std()
    ratio = np.nan_to_num(clf_s/clf_fnsf) * (clf_s > 5).astype(np.float)
    ratio_sort = np.argsort(ratio)[::-1]
    top_k = 100
    seen = {}
    i = -1
    while len(seen) < top_k and i < (ratio_sort.shape[0]-1) :
        i+=1

        idx = ratio_sort[i]
        conf_name = all_venues[idx//years_per_conf]
        if conf_name in seen: 
            continue
        year = min_year + (idx % years_per_conf)*YEAR_BLOCKS
        #if len(seen) > 25:
        #print(conf_name,clf_s[idx],clf_fnsf[idx])
        print("'"+conf_name+"'",end=',')#,year#,clf_s[idx],clf_fnsf[idx])
        seen[conf_name] = year
        

In [ ]:
top_k = 100
i = -1
j = 0
seen = {}
while j < top_k:
    i += 1
    idx = conf_ord[-(i+1)]
    conf_name = all_venues[idx//years_per_conf]
    if conf_name in seen: #or (idx%span_years) < 40 or (idx%span_years) > 45:
        continue
    j+=1
    conf_score = clf[idx]
    seen[conf_name] = 1
    print('{:40s}\t{}\t\t{:.3f}\t{:.2f}'.format(conf_name[:38],min_year + (idx % years_per_conf),100*conf_score,(conf_score-ms)/ss))

In [ ]:
[_ for _ in all_venues if 'Nano to Macro' in _] #SC INFOCOM CCCG SIGCSE HotOS WWOS 
#XP7.52 Workshop on Database Theory      	1978		885.024	10.96
#ACM SIGMOD Workshop on Research Issues  	1985		869.130	10.75

In [ ]:
[_ for _ in all_venues if 'nature'.lower() in _.lower()]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

conf_fix = csv2dict_str_str('name_alias.csv')
conf_fix.update({'International Journal of Computer Vision': 'IJCV',
             'Conference on Designing Interactive Systems':'DIS',
            'IEEE Trans. Pattern Anal. Mach. Intell.' : 'PAMI',
            'Robotics: Science and Systems':'RSS',
            'SIGMOD Conference': 'SIGMOD',
            'I. J. Robotics Res.': 'IJRR',
            'Symposium on Computer Animation':'SCA',
            'Comput. Graph. Forum':'CGF',
            'ACM Trans. Comput.-Hum. Interact.': 'TOCHI',
            'Theory of Computing': 'ToC',
                 'Conference on Computational Complexity':'CCC',
            'Comput. Graph. Forum': 'CGF',
            'APPROX-RANDOM':'APPROX-RANDOM',
                 'IEEE Trans. Med. Imaging': 'IEEE T-MI'
           })

for clf in [clf_gold]:
    plt.style.use('fivethirtyeight')

    plt.style.use('seaborn-whitegrid')

    conf_choice2 = ['SIGGRAPH','AAAI','NIPS','CVPR','ICRA','ICML','ICCV','ECCV', 'I. J. Robotics Res.',
                    'WACV','CHI','3DV','HRI',  'AAMAS','IJCAI',
                   'ISER','Robotics: Science and Systems','IROS','CoRL','ICLR']
    #conf_choice2 = ['CVPR','ECCV','ICCV','International Journal of Computer Vision','Robotics: Science and Systems']
    #conf_choice2 = ['SIGMOD Conference','VLDB','ICDT','KDD','ACM Trans. Inf. Syst.','PODS']
    #conf_choice2 = ['ACL','EMNLP','EACL']

    conf_choices = [conf_choice2, 
                    ['STOC','FOCS','SODA','Conference on Computational Complexity','COLT','APPROX-RANDOM'],
                    ['UAI','AAAI','IJCAI','ICML','NIPS','AISTATS'],
                    ['ECCV','ICCV','CVPR','WACV','International Journal of Computer Vision','IEEE Trans. Pattern Anal. Mach. Intell.'],
                    ['Robotics: Science and Systems','IROS','ICRA','WAFR','HRI','I. J. Robotics Res.'],
                    ['SIGGRAPH','SIGGRAPH Asia','I3D','ACM Trans. Graph.','Symposium on Computer Animation','Comput. Graph. Forum'],
                    ['SIGIR','JCDL','CIKM','KDD','WWW','SIGMOD Conference'],
                    ['CHI','Conference on Designing Interactive Systems','CSCW','UbiComp','UIST','ICWSM'],
                    ['IEEE Trans. Med. Imaging','MICCAI','IPMI','ISBI','NeuroImage','IPCAI'],
                    ['EC','WINE','SIGecom Exchanges','ACM Trans. Economics and Comput.','SAGT'],
                    ['ACCV','BMVC','FGR','WACV','CRV']
                   ]
    cm = plt.cm.get_cmap('tab20')

    labels = ['All','Theory','AI/ML','Vision','Robotics','Graphics','IR','HCI','Medical','Economics','Vision #2']
    #conf_choices = [['Robotics: Science and Systems','IROS','ICRA','CoRL','WAFR','HRI','ISER']]
    for conf_choice2,l in zip(conf_choices,labels):


        #conf_choice2 = 
        conf_choice3 = []
        vs = clf.std()
        for conf in conf_choice2:
            idx = conf_idx[conf]
            s = max(clf[years_per_conf*idx+(2014-1970)//YEAR_BLOCKS:years_per_conf*idx+(2019-1970)//YEAR_BLOCKS])#max(clf[span_years*idx:span_years*(idx+1)]) ##s = clf[span_years*idx+2015-1970]#
            conf_choice3.append((s,conf))
        if l == 'All':
            plt.figure(figsize=(18,10))
        else:
            plt.figure(figsize=(3*2.7,2*2.7))
        index = 0
        for s,conf in sorted(conf_choice3,reverse=True):
            idx = conf_idx[conf]
            if conf in conf_fix:
                conf = conf_fix[conf]
            if l == 'All':
                line = (clf[years_per_conf*idx:years_per_conf*(idx+1)]/vs)[:-1]
                line = [float('nan') if x==0 else x for x in line]

                _ = plt.plot((np.arange(years_per_conf-1)*YEAR_BLOCKS +1970 ),line,marker='.',markersize=15,label='{:9s} ({:.1f})'.format(conf,s/vs),lw=7,alpha=0.9,c=cm(index))
            else:
                line = (np.maximum(0,clf[years_per_conf*idx:years_per_conf*(idx+1)]/vs))[:-1]
                line = [float('nan') if x==0 else x for x in line]

                _ = plt.plot((np.arange(years_per_conf-1)*YEAR_BLOCKS +1970 ),line,alpha=0.9,marker='.',markersize=15,label='{} ({:.1f})'.format(conf,s/vs),lw=5)
            index += 1

        plt.grid(True)
        plt.xlabel('year',fontsize=26)
        plt.ylabel('venue scores',fontsize=26)
        #plt.title('WITH Temporal Adjustment')
        plt.title(l,fontsize=32) #+ ' (with size normalization)'
        #plt.title(l+ ' (with size normalization)',fontsize=32) #'

        plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
        if l == 'All':
            plt.legend(loc=3,fancybox=True, framealpha=0.8, borderpad=1,frameon=True,markerfirst=True,prop={'stretch':0,'size':18,'family': 'monospace','weight':500})
        else:
            plt.legend(loc=3,fancybox=True, framealpha=0.8, borderpad=1,frameon=True,markerfirst=True,fontsize=18)
              #plt.ylim(0,16.)
        plt.ylim(bottom=0.05) # hide the 0 label, effectively. can also pad x axis below
        plt.xlim(1970,2020)
        plt.xticks(np.arange(1970,2020,10),[str(_) for _ in np.arange(1970,2020,10)],fontsize=22)
        plt.yticks(fontsize=22)
        #plt.gca().tick_params('x',pad=10)
        plt.tight_layout()
        plt.savefig(l.split('/')[0] + '_t_year_size.pdf',facecolor='w',edgecolor='w')


        #plt.show()

    #plt.show()
clf = clf_gold


In [ ]:
for style in []:##plt.style.available:#:
    plt.style.use(style)
    plt.figure(figsize=(12,8))
    for s,conf in sorted(conf_choice3,reverse=True):
        idx = conf_idx[conf]
        weights = [clf_gold[years_per_conf*idx + yr]/vs for yr in offset_years]
        _ = plt.plot(np.arange(min_year,max_year+1),weights,label=conf,lw=5)
    plt.grid(True)
    plt.xlabel('year')
    plt.ylabel('value')
    plt.title(style)
    plt.legend()

In [ ]:
if Xauth is None:
    count_vecs = {}
    paper_vecs = []
    for paper in all_papers:
        tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
        n = len(authors)
        j = years_per_conf*conf_idx[venue] + (year-min_year)//YEAR_BLOCKS

        if n not in count_vecs:
            author_scores = np.ones(n) #1/(np.arange(n)+1) 
            #author_scores[-1] = author_scores[0]
            author_score_sum = author_scores.sum()
            #author_scores /= author_score_sum
            count_vecs[n] = author_scores #/ author_score_sum
        else:
            author_scores = count_vecs[n]
            paper_vecs.append([(name_idx[a],j,v) for a,v in zip(authors,author_scores)])

In [ ]:
if Xauth is None or Xauth.shape[1] != years_per_conf*n_confs:
    import scipy.sparse
    Xauth = scipy.sparse.dok_matrix((n_auths,years_per_conf*n_confs))
    xdict = {}
    if False:
        for paper in all_papers:
            tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
            n = len(authors)
            j = years_per_conf*conf_idx[venue] + (year-min_year)//YEAR_BLOCKS
            for ai in range(n):#zip(count_vecs[n],authors):
                i = name_idx[authors[ai]]
                #xdict[(i,j)] = 1/n + xdict.get((i,j),0)
                xdict[(i,j)] = count_vecs[n][ai] + xdict.get((i,j),0)

    else:
        for paper_vec in paper_vecs:
            for i,j,v in paper_vec:
                xdict[(i,j)] = v + xdict.get((i,j),0)

    Xauth._update(xdict)
            
    Xauth = scipy.sparse.csr_matrix(Xauth)

In [ ]:
paper_counts = np.zeros(n_auths)
paper_counts_c = np.zeros(n_confs)
paper_counts_year = {}#np.zeros(n_auths)
paper_counts_year_counter = np.zeros((max_year-min_year+1))

for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    paper_counts_c[conf_idx[venue]] += 1
    paper_counts_year_counter[year-min_year] += 1
    for a in authors:
        i = name_idx[a]
        paper_counts[i] += 1
        paper_counts_year[(a,year)] = 1+ paper_counts_year.get((a,year),0)

In [ ]:
cmu_uni = pd.read_csv('other_ranks/cmu_faculty.csv')
cmu_uni = cmu_uni.fillna('Other')
cmu_uni = cmu_uni[cmu_uni.dept == 'RI']
#print(list(cmu_uni.name))
uni_names = list(cmu_uni.name)#['Xuemin Shen','H. Vincent Poor','Kang G. Shin','Mohamed-Slim Alouini','Lajos Hanzo']#list(cmu_uni.name)
#uni_names = list(faculty_affil[faculty_affil.affiliation == 'Johns Hopkins University'].name)
print(len(uni_names))
conf_counts = {}
conf_counts_value = {}

for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    n = len(authors)
    if year < 2000:
        continue
    for a in authors:
        if a in uni_names:
            conf_counts[venue] = 1/n + conf_counts.get(venue,0)
            conf_counts_value[venue] = clf_gold[years_per_conf*(conf_idx[venue]) + (year-min_year)//YEAR_BLOCKS]/n + conf_counts_value.get(venue,0)
conf_counts_value = {k: v/conf_counts[k] for k,v in conf_counts_value.items()}
ri_fav_confs = [(conf_counts[_[1]]*conf_counts_value[_[1]],_[1],conf_counts[_[1]],conf_counts_value[_[1]]) for _ in sorted([(v,k) for k,v in conf_counts.items() if v > 0],reverse=True)]
[_[1] for _ in sorted(ri_fav_confs,reverse=True)]

In [ ]:
year_filter = np.zeros_like(clf_gold).reshape((-1,years_per_conf))
#year_filter[:-10] = 1
confs_to_filter = ['ECCV','ICCV','CVPR','WACV','International Journal of Computer Vision',
                   'IEEE Trans. Pattern Anal. Mach. Intell.',
                   'Robotics: Science and Systems','IROS','ICRA','WAFR','HRI','I. J. Robotics Res.',
                   '3DV','BMVC','ACCV','ISRR','ISER','NIPS','SIGGRAPH','AAAI','Humanoids','SIGGRAPH Asia']
confs_to_filter = ri_fav_confs
best_ri_confs = []
for conf in confs_to_filter:
    if conf[-2] >= 1.0:
        best_ri_confs.append(conf)
        #print(conf)
#        year_filter[conf_idx[conf[1]],-7:] = 1
#year_filter[:,-15:] = 1

for thing in sorted([(_[-1],_[:-1]) for _ in best_ri_confs],reverse=True):
    print(thing)
year_filter = np.ones_like(clf_gold).reshape((-1,years_per_conf))

total_scores = Xauth.dot(clf_gold * year_filter.reshape((-1)))


In [ ]:
page_rank_people = pickle.load(open('new_pagerank_people.pkl','rb'))
page_rank_confs = pickle.load(open('new_pagerank_conf.pkl','rb'))
page_rank_clf = np.repeat(page_rank_confs,years_per_conf)
page_rank_scores = Xauth.dot(page_rank_clf)


In [ ]:
df_corr = pd.read_csv('other_ranks/correlation_cleaned.csv')
df_corr = df_corr.drop(columns=[_ for _ in df_corr.columns if 'Unnamed' in _])
df_corr = df_corr.drop(columns=['pms','n_papers'])
df_corr = df_corr.rename(columns={'totals': 'venue_score', 'csrp': 'csr_pubs','csrpn': 'csr_adj','gcite': 'influence'})
df_corr = df_corr[['name','papers', 'citations', 'h-index',
       'i10','csr_pubs', 'csr_adj','venue_score','influence']]
df_corr = df_corr.dropna('index')
df_corr.index = df_corr.name
df_corr['pagerankA'] = df_corr.venue_score
df_corr['pagerankC'] = df_corr.venue_score

ts = total_scores.std()
tm = total_scores.mean()
for name in df_corr.name:
    if name in name_idx:
        idx = name_idx[name]
        df_corr.loc[name,'venue_score'] = total_scores[idx]
        df_corr.loc[name,'pagerankA'] = page_rank_people[idx]
        df_corr.loc[name,'pagerankC'] = page_rank_scores[idx]
        #print(name,df_corr.loc[name,'papers'],paper_counts[idx])
        df_corr.loc[name,'papers'] = paper_counts[idx]


print(df_corr.corr('spearman').loc['influence','venue_score'],df_corr.corr('kendall').loc['influence','venue_score'],df_corr.corr('spearman').loc['h-index','venue_score'])
#if clfn == clfs_test.shape[-1]:
df_corr[['papers','citations','h-index','i10','csr_pubs','venue_score','pagerankA','pagerankC','influence']].corr('spearman')

In [ ]:
df_merged = pd.read_csv('other_ranks/faculty_affil_scholar.csv')
fuzzythresh=0.9 #0.9
df_merged.loc[df_merged.fuzzyscore > fuzzythresh,'dblpname'] = df_merged[df_merged.fuzzyscore > fuzzythresh].fuzzyname
df_merged.loc[df_merged.fuzzyscore > fuzzythresh,'dblpexists'] = 1
df_merged = df_merged[df_merged.dblpexists == 1]
df_merged['venue_score'] = np.ones_like(df_merged.dblpexists)
df_merged['pagerankA'] = np.ones_like(df_merged.dblpexists)
df_merged['pagerankC'] = np.ones_like(df_merged.dblpexists)
df_merged['papers'] = np.ones_like(df_merged.dblpexists)

ts = total_scores.std()
tm = total_scores.mean()
df_merged = df_merged.set_index(df_merged.dblpname)
seen_map = {}
for name in df_merged.index:
    if name in name_idx:
        idx = name_idx[name]
        seen_map[name] = 1
        df_merged.loc[name,'venue_score'] = total_scores[idx]
        df_merged.loc[name,'pagerankA'] = page_rank_people[idx]
        df_merged.loc[name,'pagerankC'] = page_rank_scores[idx]
        df_merged.loc[name,'papers'] = paper_counts[idx]
df_merged = df_merged.drop(columns=['Unnamed: 0','First Name','Last Name','Sholar link','Rank (Full, Associate, Assistant, Other)','Full Name','University_y','University_x','Unnamed: 11',"ID",'fuzzyname','dblpexists','fuzzyscore','UniversityID'])

In [ ]:
df_msar_filt = df_msar[df_msar.dblp_name != 'NotAConf'].copy()#.sort_values('H',ascending=False)
scores = []
paperss = []
pagerankc = []
for row in df_msar_filt.itertuples():
    conf = row[-1]
    if conf in conf_idx:
        idx = conf_idx[conf]
        weights = [clf_gold[years_per_conf*idx + yr] for yr in offset_years[1984-min_year:]]
        #scores.append(result_clf[years_per_conf*idx + offset_years[2014-min_year]])
    
        scores.append(np.max(np.array([w for w in weights])))
        pagerankc.append(page_rank_confs[idx])
        paperss.append(paper_counts_c[idx])
    else:
        print(conf)
        scores.append(-1)
        pagerankc.append(-1)
        paperss.append(-1)
    #print(scores[-1],weights)

In [ ]:
new_datas = []
for row in faculty_affil.itertuples():
    if row[1] in name_idx: #and row[1] not in seen_map:
        idx = name_idx[row[1]]
        seen_map[row[1]] = 1
        new_data = {}
        new_data['dblpname'] = row[1]
        #new_data['index'] = row[1]
        new_data['school'] = row[2].strip().rstrip()
        new_data['venue_score'] = total_scores[idx]
        new_data['pagerankA'] = page_rank_people[idx]
        new_data['pagerankC'] = page_rank_scores[idx]
        new_data['papers'] = paper_counts[idx] if total_scores[idx] > 0 else 0
        new_data['authors'] = 1 if total_scores[idx] > 0 else 0
        
        new_datas.append(new_data)
        #df_merged = df_merged.append([row[1],np.nan,np.nan,np.nan,row[2],row[1],(total_scores[idx]-tm)/ts])
df_csr_to_add = pd.DataFrame(new_datas)
#df_csr_to_add = df_csr_to_add.set_index('dblpname')
df_csr_to_add = df_csr_to_add.set_index('dblpname')
#df_merged = pd.concat([df_merged,df_csr_to_add])
print(df_merged.shape)
#print(faculty_affil.shape)
df_t10 = df_merged[df_merged['t10-index'].notna()]


In [ ]:
for row in df_csr_to_add.sort_values('venue_score',0,False).itertuples():
    print('{},{},{:.0f},{:.0f}'.format(row[0],row[-2],row[-1],row[-3]))


In [ ]:
print_school_df = df_csr_to_add.groupby('school').sum()
print_school_df['size_normed'] = np.array(print_school_df.venue_score)/np.maximum(1,np.log(print_school_df.authors+1))
print_school_df.sort_values('size_normed',0,False).to_csv('schools.csv')
#total_scores[name_idx['Aaron Clauset']],total_scores[name_idx['Martial Hebert']]

In [ ]:
school_vals = df_t10.groupby('school').aggregate('sum').sort_values('venue_score',0,False)
t10_schools = school_vals.sort_values('t10-index',0,False)
school_vals.corr('spearman')

In [ ]:
try:
    df_msar = pd.read_csv('other_ranks/traditional_conf_scores.csv')
except:
    from fuzzywuzzy import process, fuzz
    with open('other_ranks/msar.json') as fp:
        msar = json.load(fp)
    df_msar = pd.DataFrame(msar)
    dblp_conf_name = []
    matchable_names = [fuzz._process_and_sort(n,False) for n in all_venues]
    for row in df_msar.itertuples():
        #print(row[2],row[-1])
        try:
            if row[-1] in conf_idx:
                dblp_conf_name.append(row[-1])
            elif row[2] in conf_idx:
                dblp_conf_name.append(row[2])
            elif len(row[2].split('/')) > 1:
                found = False
                for subname in row[2].split('/'):
                    if found == False and subname in conf_idx:
                        dblp_conf_name.append(subname)
                        found = True
                if found == False:
                    raise
            elif len(row[2].split('(')) > 1:
                substr = row[2].split('(')
                found = False
                for subname in [substr[0],substr[1][:-1]]:
                    if found == False and subname in conf_idx:
                        dblp_conf_name.append(subname)
                        found = True
                if found == False:
                    raise
            else:
                raise
        except:
            bestshort,bestlong = None,None
            if row[2] != None:
                matcher = fuzz.SequenceMatcher(None, fuzz._process_and_sort(row[2],False))
                n3s = []
                for n2 in matchable_names:
                    matcher.set_seq2(n2)
                    n3s.append(matcher.ratio())
                v=np.argmax(n3s)
                bestshort = (all_venues[v],n3s[v])
                #print(bestshort[1:],end='\t')
            if row[-1] != None:
                matcher = fuzz.SequenceMatcher(None, fuzz._process_and_sort(row[-1],False))
                n3s = []
                for n2 in matchable_names:
                    matcher.set_seq2(n2)
                    n3s.append(matcher.ratio())
                v=np.argmax(n3s)
                bestlong = (all_venues[v],n3s[v])
                #print(bestlong[1:],end='\t')
            if bestlong and bestlong[-1] > 0.96:
                dblp_conf_name.append(bestlong[0])
                #print(bestlong,row)
            elif bestshort and bestshort[-1] > 0.96:
                dblp_conf_name.append(bestshort[0])
                #print(bestshort,row)
            else:
                #print(bestlong,bestshort,row[2],row[-1])
                dblp_conf_name.append('NotAConf')
    df_msar['dblp_name'] = dblp_conf_name
    df_msar.to_csv('other_ranks/traditional_conf_scores.csv')
    #ILPS/ISLP/NACLP/SLP
    #DISC(WDAG)


offset_years = [i//YEAR_BLOCKS for i in range(span_years)]

In [ ]:
paper_counts_c.shape
counter = 0
for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    if venue == 'Internet Measurement Workshop':
        counter += 1
counter,paper_counts_c[conf_idx['IMC']]


In [ ]:
df_msar_filt['venue_scores'] = scores
df_msar_filt['pagerankc'] = pagerankc
df_msar_filt['papers'] = paperss
df_msar_filt['cp'] = df_msar_filt.citations/df_msar_filt.papers


df_msar_filt = df_msar_filt[~df_msar_filt.dblp_name.duplicated()].copy()
df_msar_filt = df_msar_filt[~df_msar_filt.venue_scores.duplicated()].copy()
#df_msar_filt['h-approx'] = 0.54*np.sqrt(df_msar_filt.citations)

df_msar_filt.shape

In [ ]:
df_msar_filt[df_msar_filt.category == 'Computer Vision'].sort_values('cp',0,False)

In [ ]:
thresh = 15
df_msar_filt = df_msar_filt[['papers','citations','H','pagerankc','cp','venue_scores']]
print(df_msar_filt[df_msar_filt.H > thresh].shape)
df_msar_filt[df_msar_filt.H > thresh].corr('spearman')


In [ ]:
bp2 = pd.read_csv('other_ranks/uni_rank_bp.csv')
times = pd.read_csv('other_ranks/uni_rank_times.csv')

srf2 = pd.read_csv('other_ranks/uni_rank_mergedscholar.csv')
st2 = pd.read_csv('other_ranks/uni_rank_st.csv')
qt2 = pd.read_csv('other_ranks/uni_rank_qt.csv')
sr2 = pd.read_csv('other_ranks/uni_rank_sr.csv')
pr2 = pd.read_csv('other_ranks/uni_rank_pr.csv')
cm2 = pd.read_csv('other_ranks/uni_rank_cs.csv')
usn2 = pd.read_csv('other_ranks/uni_rank_usn.csv')
df_csr = pd.read_csv('other_ranks/ranks.csv')

pr2.USN2010 = pr2.USN2010.map(lambda x: int(x) if x.isnumeric() else np.nan)


In [ ]:
datasets = [
    [(row[2],row[1]) for row in st2.itertuples()],
    [(row[2],row[1]) for row in times.itertuples()],
    [(row[2],row[1]) for row in qt2.itertuples()],
    [(row[6],row[1]) for row in pr2.itertuples()],
    [(row[2],row[1]) for row in cm2.itertuples()],
    [(row[2],row[1]) for row in sr2.itertuples()],
    [(row[-1],row[2]) for row in srf2.itertuples()],

    [(row[0],idx+1) for idx,row in enumerate(t10_schools.itertuples())],
    [(row[2],row[1]) for row in df_csr.itertuples()],
    [(row[0],idx+1) for idx,row in enumerate(school_vals.itertuples())],
    [(row[0],idx+1) for idx,row in enumerate(school_vals.sort_values('pagerankA',0,False).itertuples())],
    [(row[0],idx+1) for idx,row in enumerate(school_vals.sort_values('pagerankC',0,False).itertuples())],
    [(row[0],idx+1) for idx,row in enumerate(school_vals.sort_values('Citations',0,False).itertuples())],
    [(row[0],idx+1) for idx,row in enumerate(school_vals.sort_values('papers',0,False).itertuples())],

    [(row[2],row[1]) for row in bp2.itertuples()],
    [(row[6],row[4]) for row in pr2.sort_values('NRC95',ascending=True).itertuples() ],
    [(row[6],row[3]) for row in pr2.sort_values('USN2010',ascending=True).itertuples() if np.isfinite(row[3]) ],
    [(row[2],row[1]) for row in usn2.itertuples()]
]
dataset_names = ['Shanghai','Times','QS','Prestige','CSMetrics',
                 'ScholarRank','ScholarRankFull','t10Sum','CSRankings','Mine','pageranka','pagerankc','citations','papers','BestPaper','NRC95',"USN10",'USN18']
n_datasets = len(datasets)


In [ ]:
#school_vals.to_csv('test3.csv')

In [ ]:
corr_matrix = np.zeros((n_datasets,n_datasets))
count_matrix = np.zeros((n_datasets,n_datasets))
name_datasets = [ [v[0] for v in d] for d in datasets ]
all_names = sorted(list(set(sum(name_datasets,[]))))
all_vec = [sum([name in d for d in name_datasets])>=(len(datasets)-4) for name in all_names]
subset_names = [name for name,vec in zip(all_names,all_vec) if vec]
subset_names,len(subset_names)
import scipy.stats as stats
for i in range(n_datasets):
    inames = [u[0] for u in datasets[i]]
    for j in range(i,n_datasets):
        try:
            jnames = [u[0] for u in datasets[j]]

            #exist_1 = [((ni in subset_names) and (ni in jnames)) for ni in inames]
            #exist_2 = [((nj in subset_names) and (nj in inames))for nj in jnames]
            exist_1 = [((True) and (ni in jnames)) for ni in inames]
            exist_2 = [((True) and (nj in inames))for nj in jnames]

            d1 = np.array(datasets[i])[exist_1]
            d2 = np.array(datasets[j])[exist_2]
            v1 = d1[:,1].astype(np.float)
            v2 = np.array([d2[np.where(d2[:,0] == name)[0][0],1] for name in d1[:,0]]).astype(np.float)
            c = stats.spearmanr(v1,v2)[0]
            corr_matrix[i][j] = c
            corr_matrix[j][i] = c
            count_matrix[i][j] = len(v1)
            count_matrix[j][i] = len(v2)
        except:
            print(len(v1),len(v2))
        #print(v1,v2)

In [ ]:
np.set_printoptions(precision=2)
print(corr_matrix)
print('mean best')
for s,n in sorted([(s,n) for n,s in zip(dataset_names,corr_matrix.mean(1))],reverse=True):
    print('{:30s}\t{:.3f}'.format(n,s))
print('\n usnews best')
for s,n in sorted([(s,n) for n,s in zip(dataset_names,corr_matrix[-1])],reverse=True):
    print('{:30s}\t{:.3f}'.format(n,s))
print('\n names')

for n in dataset_names:
    print(n)

In [ ]:
plt.style.use('seaborn-whitegrid')

In [ ]:
from collections import Counter,defaultdict
def di():
    return defaultdict(float)
author_by_year = defaultdict(di)
for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    n = len(authors)
    for a in authors:
        author_by_year[a][year] += clf_gold[years_per_conf*conf_idx[venue] + offset_years[year-min_year]]/n

In [ ]:
from scipy.ndimage.filters import gaussian_filter1d
import matplotlib.gridspec as gridspec
gs = gridspec.GridSpec(4, 4)
plt.figure(figsize=(12,12))
#example_names = ['Takeo Kanade','Martial Hebert','Christopher G. Atkeson','Howie Choset','Deva Ramanan','Jessica K. Hodgins'] #,'Pieter Abbeel'
#Pearl, Guibas, Karp, Kanade, Jordan, Woodruff
example_names = ['Judea Pearl','Leonidas J. Guibas','Richard M. Karp','Takeo Kanade','Michael I. Jordan','Elisa Bertino']
#example_names = ['Christopher Scaffidi','Humphrey Hu','Keenan Crane']
#example_names = ['Takeo Kanade','Michael I. Jordan','Richard M. Karp','Leonidas J. Guibas','David P. Woodruff', 'Judea Pearl']
#example_names = ['Christopher G. Atkeson','Oliver Kroemer','Jeannette Bohg','Mark R. Cutkosky','Doug L. James','Allison M. Okamura','Edward H. Adelson','Pulkit Agrawal','Phillip Isola','Alberto Rodriguez','Russ Tedrake']
#example_names = ['Dorsa Sadigh','Chelsea Finn','Oliver Kroemer','Jeannette Bohg','Phillip Isola','Alberto Rodriguez']
#example_names = ['Christopher G. Atkeson','Mark R. Cutkosky','Doug L. James','Allison M. Okamura','Edward H. Adelson','Russ Tedrake']
#example_names = ['Martial Hebert','Jessica K. Hodgins','Christopher G. Atkeson','Katia P. Sycara','Deva Ramanan','Pieter Abbeel']
ri_names = list(set([aliasdict.get(row[1],row[1]) for row in pd.read_csv('other_ranks/cmu_faculty.csv').itertuples() if row[2] != 'c']))
#example_names = [_[1] for _ in sorted([(total_scores[name_idx[i]],i,year_span[name_idx[i]]) for i in ri_names if i in name_idx],reverse=True)[:6]]
plt.subplot(gs[2:4, 1:3])

for example_name in example_names:
    example_value = np.zeros(max_year+1-min_year)
    years = author_by_year[example_name]
    yrs = [_ for _ in years.keys() if _  > 0]
    start_year = min(yrs)
    end_year = max(yrs)
    span = end_year - start_year
    start_year,end_year,span
    for y,v in years.items():
        example_value[y-min_year] += v
            
    plt.plot(np.arange(min_year,max_year+1)[:-1],gaussian_filter1d(example_value[:-1], sigma=4.5),label=example_name,lw=4)
    #plt.plot(gaussian_filter1d(example_value[:span], sigma=2),label=example_name )

#plt.plot((val_by_year/v_count),label='average author')
plt.ylabel('annual value (4.5 sigma smoothing)')
#plt.xlabel('year since first publication')
plt.xlabel('year',fontsize=18)
plt.title('venue scores')
plt.legend(bbox_to_anchor=(0.5, 1.4),loc=9,fancybox=True, framealpha=0.8, borderpad=1,frameon=True,ncol=3,fontsize=16)
plt.grid(True)
plt.subplot(gs[:2, :2])
for example_name in example_names:
    example_value = np.zeros(max_year+1-min_year)
    for yr in range(min_year,max_year+1):
        if (example_name,yr) in paper_counts_year:
            example_value[yr-min_year] += paper_counts_year[example_name,yr]
    plt.plot(np.arange(min_year,max_year+1)[:-1],gaussian_filter1d(example_value[:-1], sigma=4.5),label=example_name,lw=4)
    #plt.plot(gaussian_filter1d(example_value[:span], sigma=2),label=example_name )

#plt.plot((val_by_year/v_count),label='average author')
plt.ylabel('annual value (4.5 sigma smoothing)')
#plt.xlabel('year since first publication')
plt.xlabel('year',fontsize=18)
plt.title('paper count')
#plt.legend(loc=4,fancybox=True, framealpha=0.8, borderpad=1,frameon=True,ncol=2,fontsize=12)
plt.tight_layout()
plt.grid(True)
plt.subplot(gs[:2, 2:])

for example_name in example_names:
    example_value = np.zeros(max_year+1-min_year)
    for yr in range(min_year,max_year+1):
        if (example_name,yr) in paper_counts_year:
            example_value[yr-min_year] += paper_counts_year[example_name,yr]
    plt.plot(np.arange(min_year,max_year+1)[:-1],1e4*gaussian_filter1d((example_value/paper_counts_year_counter)[:-1], sigma=4.5),label=example_name,lw=4)
    #plt.plot(gaussian_filter1d(example_value[:span], sigma=2),label=example_name )

#plt.plot((val_by_year/v_count),label='average author')
plt.ylabel('annual value (4.5 sigma smoothing)')
#plt.xlabel('year since first publication')
plt.xlabel('year',fontsize=18)
plt.title('paper count (norm by annual paper totals)')
#plt.legend(fancybox=True, framealpha=0.8, borderpad=1,frameon=True,ncol=2,fontsize=12)
plt.tight_layout()
plt.grid(True)
plt.savefig('fame4.pdf',facecolor='w',edgecolor='w')

In [ ]:
val_by_year = np.zeros(50)
v_count = np.zeros(50)
for a in author_by_year:
    vector = author_by_year[a]
    yrs = vector.keys()
    start_year_a = min(yrs)
    end_year_a = max(yrs)
    span = end_year_a - start_year_a
    if span < 85:
        for y in vector:
            v_count[y-start_year_a] += 1
            val_by_year[y-start_year_a] += vector[y]

In [ ]:

plt.figure()
plt.plot((val_by_year/v_count)[:-2])
plt.title('Aging Curve for Authors')
plt.xlabel('years since first publication')
plt.ylabel('average annual value generated')
plt.xlim(0,50)
#plt.ylim(0,2)
plt.ylim(bottom=0)
plt.grid(True)
plt.tight_layout()
plt.savefig('age_curve.pdf',facecolor='w',edgecolor='w')

In [ ]:
total_values = {k: sum(v.values()) for k,v in author_by_year.items()}

In [ ]:
total_value_list = sorted([(v,k) for k,v in total_values.items()],reverse=True)

In [ ]:
total_value_list[:500]

In [ ]:
curious_names = [
    'Aditya Dhawale',
    'Tesca Fitzgerald',
    'Adam W. Harley',
    "Xiaolong Wang 0004",
    "Judy Hoffman",
    "Paris Siminelakis",
    "Roie Levin",
    "Leonid Keselman",
    "Rick Goldstein",
    "Nicholas Rhinehart",
    "Vincent Sitzmann",
    "Siddharth Ancha",
    "Xingyu Lin",
    "Humphrey Hu",
    "David F. Fouhey",
    "Chelsea Finn",
    "Dinesh Jayaraman",
    "Wen Sun 0002",
    "Lerrel Pinto",
    "Justin Johnson",
    "Amir Roshan Zamir",
    "Dominik Peters",
    "Jonathan T. Barron",
    "Dorsa Sadigh",
    "Derek Hoiem",
    "Vaggos Chatziafratis",
    "Brian Okorn",
    "David Held"
]

In [ ]:
ri_names = list(set([aliasdict.get(row[1],row[1]) for row in pd.read_csv('other_ranks/cmu_faculty.csv').itertuples() if row[2] == 'RI']))

In [ ]:
uni_names = np.array(['Artur Dubrawski','Artur W. Dubrawski'])
uni_names = list(uni_names)
cmu_scores = []
for name in uni_names:
    if name in name_idx:
        score = total_scores[name_idx[name]]
        cmu_scores.append(((score-tm)/ts,name))
for s,p in sorted(cmu_scores,reverse=True):
    print('{:30s}\t\t{:.3f}'.format(p,s))


In [ ]:
for v,k in sorted([(v,k) for k,v in conf_counts.items() ],reverse=True):
    print('{:30s}\t{}'.format(k[:30],round(v)))

In [ ]:
prof_conf_counts = {}
for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    n = len(authors)
    if year < 2004:
        continue
    for a in authors:
        if a in uni_names:
            prof_conf_counts[venue] = [a] + prof_conf_counts.get(venue,[])

In [ ]:
confs_of_interest =[]# ['CVPR','ICCV','ECCV']
for conf_of_interest in confs_of_interest:
    unique_set = {k: 0 for k in set(prof_conf_counts[conf_of_interest])}
    for k in prof_conf_counts[conf_of_interest]:
        unique_set[k] += 1
    for v,k in sorted([(v,k) for k,v in unique_set.items() ],reverse=True):
        print('{:30s}\t{}'.format(k[:30],round(v)))

In [ ]:
good_confs = [k for k,v in conf_counts.items() if v > 1]
faculty_lookup = {_[1]:_[2] for _ in faculty_affil.itertuples()}

In [ ]:
filter_confs = np.zeros_like(clf_gold) 
for conf in good_confs:
    filter_confs[conf_idx[conf]*years_per_conf:(conf_idx[conf]+1)*years_per_conf] = 1
year_span = (auth_years[:,1] - auth_years[:,0]) + 1
ri_scores_total = (Xauth.dot(clf_gold * filter_confs))
ri_scores = ri_scores_total * (auth_years[:,0] >= 2010)

In [ ]:
best_ri_confs = [_ for _ in np.argsort(clf_gold * filter_confs)[::-1] if filter_confs[_]]
for idx in best_ri_confs:
    year = idx%years_per_conf
    if (year+1970) < 2016:
        continue
    print('{:45s}\t{:.2f}\t{}'.format(all_venues[idx//years_per_conf][:35],clf_gold[idx],year+1970))

In [ ]:
best_ri = np.argsort(total_scores)[::-1]

In [ ]:
for i,idx in enumerate(best_ri[:500]):
    name = all_authors[idx]
    uni = faculty_lookup[name] if name in faculty_lookup else 'None'
    if uni != 'None':
        continue
    print('{},{},{:.2f}'.format(all_authors[idx],uni,total_scores[idx]))
    #print('{}\t\t{:30s}\t{:.2f}\t\t{}\t{}'.format(i,all_authors[idx],ri_scores[idx],uni,int(auth_years[idx,1])))

In [ ]:
i = 0
things2 = []
for idx in np.argsort(total_scores)[::-1][:15000]:
    name = all_authors[idx]
    uni = faculty_lookup[name] if name in faculty_lookup else 'None'
    #if uni == 'None':
    #    continue
    #if auth_years[idx,1] < 2010:
    #    continue
    i+=1
    things2.append((all_authors[idx],total_scores[idx],uni,auth_years[idx,1]-auth_years[idx,0]))
    print('{}\t\t{:30s}\t{:.2f}\t\t{}\t{}'.format(i,all_authors[idx],total_scores[idx],uni,int(auth_years[idx,1])))

In [ ]:
pd.DataFrame(things2,columns=['Name','Score','Uni','Years']).set_index('Name').to_csv('authors.csv')

In [ ]:
sorted([(total_scores[name_idx[i]]/year_span[name_idx[i]],i,year_span[name_idx[i]]) for i in ri_names],reverse=True)

In [ ]:
sorted([(total_scores[name_idx[i]],i,year_span[name_idx[i]]) for i in ri_names],reverse=True)

In [ ]:
sorted([(total_scores[name_idx[i]],i,year_span[name_idx[i]]) for i in curious_names],reverse=True)

In [ ]:
name_idx["Joshua B. Tenenbaum"],name_idx['Matthias Nießner']

In [ ]:
first_paper_year = 2013
min_pub_years = 2
cand_total = total_scores * (auth_years[:,0] >= first_paper_year).astype(np.float)* (year_span >= min_pub_years).astype(np.float) 

cand_ri = ri_scores* (auth_years[:,0] >= first_paper_year).astype(np.float) * (year_span >= min_pub_years).astype(np.float) 


cand_total_ef = cand_total/year_span
cand_ri_ef = cand_ri/year_span

print('{}\t{:30s}\t{:s}\t{:s}\t{:s}\t{:s}\t{:s}\t{:20s}'.format('Rank','Author',
                                                      'RI','Total',
                                                      'eRI','eTotal',
                                                      'Since','Affiliation'))
for num,idx in enumerate((np.argsort(cand_ri)[::-1])[:100]):
    uni = faculty_lookup.get(all_authors[idx],'None')

    print('{}\t{:30s}\t{:.1f}\t{:.1f}\t{:.1f}\t{:.1f}\t{:.0f}\t{:20s}'.format(num+1,all_authors[idx],
                                                          cand_ri[idx],cand_total[idx],
                                                          cand_ri_ef[idx],cand_total_ef[idx],
                                                          auth_years[idx,0],uni))



In [ ]:
count_of_papers = np.zeros(years_per_conf*n_confs)
for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    yr = (year-min_year)//YEAR_BLOCKS
    j = years_per_conf*conf_idx[venue] + yr
    count_of_papers[j] += 1
# safe divide
count_of_papers = np.maximum(1,count_of_papers)

In [ ]:
count_of_papers2 = count_of_papers.reshape((-1,years_per_conf))
masked_count = (count_of_papers2 * (count_of_papers2 > 1))
how_many_valid_years = (masked_count > 1).sum(1)
conf_sizes = masked_count.sum(1)/how_many_valid_years
valid_confs_for_vis = (np.isfinite(conf_sizes)) & (conf_sizes > 1)

In [ ]:
valid_confs_for_vis

In [ ]:
tmp_scores = clf_gold.reshape((-1,years_per_conf))[:,-6:-1].copy()
ranks = np.nan_to_num(np.true_divide(tmp_scores.sum(1),(tmp_scores!=0).sum(1)))

In [ ]:
short_names = {'Theory of Computing, Graduate Surveys' : 'Theory of Computing, GS',
               'IEEE Symposium on Security and Privacy':'IEEE SSP',
              'IEEE Conference on Computational Complexity': 'IEEE CCC',
              'USENIX Annual Technical Conference': 'USENIX ATC',
              'Journal of Machine Learning Research': 'JMLR',
              'Computer Communication Review' : 'SIGCOMM CCR'}
things = []
for idx in np.argsort(ranks)[::-1]:#[:500]:
    name = short_names[all_venues[idx]] if all_venues[idx] in short_names else all_venues[idx]
    paper_count = conf_sizes[idx]
    if not valid_confs_for_vis[idx]:
        continue
    if ranks[idx] < 0.1:
        continue
    things.append((name,ranks[idx],conf_sizes[idx]))
    print('{:30s}\t{:.2f}\t{:.1f}'.format(name,ranks[idx],conf_sizes[idx]))


In [ ]:
pd.DataFrame(things,columns=['Name','Score','Size']).set_index('Name').to_csv('confs.csv')

In [ ]:
ranks[conf_idx['STOC']]

In [ ]:
# needs cluster_new to be run. sorry
labels = np.load('labels.npy')
mask = np.load('mask.npy')

In [ ]:
# needs cluster_new to be run. sorry
cluster_ranks = clf_gold.reshape((-1,years_per_conf))[mask,:]
small_names = np.array([all_venues[i] for i in range(n_confs) if mask[i]])
alias_dict = csv2dict_str_str('name_alias.csv')
group_names = csv2dict_str_str('lookups.csv')
group_names = {k:v for k,v in group_names.items() if v != ''}
valid_confs_for_small_vis = valid_confs_for_vis[mask]

In [ ]:
NUM_CLUSTERS = labels.max() + 1
small_conf_counts = np.zeros(NUM_CLUSTERS)

for i in range(32):
    cluster_names = small_names[labels == i]
    for conf in cluster_names:
        small_conf_counts[i] += count_of_papers[years_per_conf*conf_idx[conf]:years_per_conf*(1+conf_idx[conf])].sum()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(32,32))
names_to_save = []

for cnti, i in enumerate(np.argsort(small_conf_counts)[::-1]):
    plt.subplot(8,4,cnti+1)
    cluster_names = small_names[labels == i]
    cluster_filtered = cluster_ranks[labels == i] * valid_confs_for_small_vis[labels == i].reshape((-1,1))
    
    cluster_scores = cluster_filtered[:,-10:].max(1)
    valid_idxs = np.argsort(cluster_filtered[:,-4:-1].mean(1))[::-1]
    for idx in valid_idxs[:6]:
        name_for_now = cluster_names[idx]
        if name_for_now in group_names:
            plt.title(group_names[name_for_now])
            #print(group_names[name_for_now],cnti,small_conf_counts[i])
        name_for_now = alias_dict[name_for_now] if name_for_now in alias_dict else name_for_now
        line = [np.nan if _ == 0 else _ for _ in cluster_filtered[idx]]
        plt.plot(min_year+np.arange(years_per_conf-1)*YEAR_BLOCKS,line[:-1],label=name_for_now)
        names_to_save.append(cluster_names[idx])
        #print(name_for_now,i)
    plt.legend(loc=3,fancybox=True, framealpha=0.8, borderpad=1,frameon=True)
    plt.tight_layout()
    #plt.ylim(0,13)
    plt.ylim(bottom=0.05)
    plt.xlim(1970,2020)
plt.savefig('big_plot4.pdf',facecolor='w',edgecolor='w')

In [ ]:
if False:
    plt.figure(figsize=(24,20))
    names_to_save = []

    #cluster_labels = ['Networks','Systems','Crypto','Robotics','Graphics','Security','Networks','Software Eng','Control','Arch','Database','Computer Vision','Design Automation','Mobile','Data Mining','Embedded','Programming Languages','Real-Time','NLP','BioMed','Comp Bio','IR','Multimedia','Communication','Theory','HCI','HPC','Econ','Computational Geometry','Machine Learning']
    for i in range(24):

        plt.subplot(6,4,i+1)
        cluster_names = conf_set[i]
        filt = np.zeros(cluster_ranks.shape[0]).astype(np.bool)
        for conf in conf_set[i]:
            if conf in small_names:
                filt[np.where(small_names == conf)[0][0]] = True
        cluster_filtered = cluster_ranks[filt]
        cluster_scores = cluster_filtered[:,-10:].max(1)
        valid_idxs = np.argsort(cluster_filtered[:,-7:-2].mean(1))[::-1][:3]
        print(cluster_names,len(valid_idxs),cluster_scores)
        for idx in valid_idxs:
            name_for_now = conf_set[i][idx]
            print(name_for_now,cluster_scores[idx])
            if name_for_now in group_names:
                plt.title(group_names[name_for_now])
            name_for_now = alias_dict[name_for_now] if name_for_now in alias_dict else name_for_now
            plt.plot(min_year+np.arange(years_per_conf-1)*YEAR_BLOCKS,cluster_filtered[idx][:-1],label=name_for_now)
            names_to_save.append(cluster_names[idx])

        plt.legend(loc=2,fancybox=True, framealpha=0.8, borderpad=1,frameon=True)
        plt.tight_layout()
        plt.ylim(0,13)
        plt.xlim(1970,2020)
    plt.savefig('big_plot2.pdf',facecolor='w',edgecolor='w')

In [ ]:
cluster_names

In [ ]:
for i in names_to_save:
    print(i)